## A. Importing Libraries

In [22]:
#import all libraries

#Data Processing
import pandas as pd

import numpy as np

#Coordinates
from geopy.geocoders import Nominatim
import requests

#Webscraping
from bs4 import BeautifulSoup
import json
from pandas.io.json import json_normalize

# Visualizations
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors
import seaborn as sns
import folium

# Clustering/Machine Learning
from sklearn.cluster import KMeans

## 1. Scrape Data

In [23]:
#GET Request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [24]:
#soupify data
soup = BeautifulSoup(data, 'html.parser')

## 2. Create Dataframe

In [25]:
#store data via lists
postalcodes = []
boroughs = []
neighborhoods = []

In [26]:
# go through text and populate lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalcodes.append(cells[0].text.rstrip('\n'))
        boroughs.append(cells[1].text.rstrip('\n'))
        neighborhoods.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell

In [27]:
#create data frame from lists
df = pd.DataFrame({"PostalCode": postalcodes,
                           "Borough": boroughs,
                           "Neighborhood": neighborhoods})

df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## 3. Cleaning Data Frame

In [28]:
#drop cells with a borough that is not assigned
df_dropna = df[df.Borough != "Not assigned"].reset_index(drop=True)
df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [29]:
# group neighborhoods in the same borough
df_grouped = df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [30]:
#Checking work
df_grouped[df_grouped["Neighborhood"] == "Not assigned"]

,PostalCode,Borough,Neighborhood


In [31]:
#checking work
df_grouped["Neighborhood"].value_counts()

Downsview                                                                 4
Don Mills                                                                 2
University of Toronto, Harbord                                            1
Wexford, Maryvale                                                         1
Bedford Park, Lawrence Manor East                                         1
                                                                         ..
Humewood-Cedarvale                                                        1
Little Portugal, Trinity                                                  1
Studio District                                                           1
Westmount                                                                 1
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale    1
Name: Neighborhood, Length: 99, dtype: int64

In [40]:
#replicate dataframe provided in example
column_names = ["PostalCode", "Borough", "Neighborhood"]
df_test = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    df_test = df_test.append(df_grouped[df_grouped["PostalCode"]==postcode], ignore_index=True)
    
df_test

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


In [41]:
df_grouped.shape

(103, 3)

## 4. Read CSV File and Merge (Part 2)

In [42]:
#Read CSV File provided by Coursera
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [43]:
#Rename Postal Code to PostalCode for later merge

coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [44]:
#Merge two data frames together on column "Postalcode"
df_new = df_grouped.merge(coordinates, on="PostalCode", how="left")
df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [45]:
#replicate coursera dataframe
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
df_test = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    df_test = df_test.append(df_new[df_new["PostalCode"]==postcode], ignore_index=True)
    
df_test

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420
